## Step 0

In [ ]:
import openai
print("Great! It worked!")

##Step 1

In [ ]:
import openai
import os

openai.api_type = "azure"
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = "2023-07-01-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")
from tech_train_functions import call_open_ai
client = None


In [ ]:
your_deployment_name = "gpt-35-turbo" #if your deployment has different name please change to the correct one
print(call_open_ai(system_message="You're an AI assistant", text="please write names of 3 animals", engine=your_deployment_name))
print(call_open_ai(system_message="You're an AI assistant", text="please write names of 4 fruits", engine=your_deployment_name))


You got the fruits and animals? Great!
Now TA4H check:

In [ ]:
import os
from tech_train_functions import analyze_TA4H
ta4h_endpoint=os.getenv("TA4H_ENDPOINT")
key=os.getenv("TA4H_KEY")

documents = [
        """
        low measurment in growth hormone.
        """
    ]
result = analyze_TA4H(ta4h_endpoint, key, documents)
print([e["normalized_text"] if e["normalized_text"] else e["text"].lower() for e in result[0]])

In [ ]:
result[0]

Got your text Analyzed? Great! Step 1 done

## Step 2 - Load Dataset and run first iteration
Load MedQA dev split:

In [ ]:
import pandas as pd
from tqdm import tqdm
pd.set_option('max_colwidth', 10000)
pd.options.mode.chained_assignment = None
df_dev = pd.read_csv("../data/MedQA_dev.csv", index_col=0)
df_train = pd.read_csv("../data/MedQA_train.csv", index_col=0)
df_dev.head(2)


In [ ]:
#create small sample for experiments:
df_dev_sample = df_dev[:40]

Now let's run our first inference!

Here we're supplying GPT with questions, options for answers, and that's all

In [ ]:
system_message = "You're a medical expert answering medical questions. \
Answer with only the correct answer and nothing else, not even the index of the answer."
sample_list = (df_dev_sample["question"]+" "+
                df_dev_sample["options"].astype(str)).tolist()
print("Input example:{}".format(sample_list[0]))


replies = []
for q in tqdm(sample_list):
    ans = call_open_ai(system_message, q, client=client)
    replies.append(ans)
df_dev_sample.loc[:,"reply"] = replies

Let's check our answers:

In [ ]:
df_dev_sample.head(3)

As you can see it's not exactly like the gold answer... we'll be using a fuzzy  match function to assess if the reply is similar to gold answer:

In [ ]:
from tech_train_functions import is_similar
df_dev_sample.loc[:,"is correct reply"] = df_dev_sample.apply(lambda row:is_similar(row["answer"], row["answer_idx"], row["reply"]), axis=1)


Let's see what is our accuracy:

In [ ]:
df_dev_sample["is correct reply"].value_counts(normalize=True)

Better than 25% random but not too good... 

##
## STEP 3 - Chain of Thought

In [ ]:
system_message_CoT = "You're a medical expert answering medical questions. Please answer step by step. Start with 'let's take it step by step'\
and end with 'the correct answer is:' and then the correct answer. Please allways end with this phrase"

sample_list = (df_dev_sample["question"]+" "+
                df_dev_sample["options"].astype(str)).tolist()
print("Input example:{}".format(sample_list[0]))


replies = []
for q in tqdm(sample_list):
    ans = call_open_ai(system_message_CoT, q, client=client)
    replies.append(ans)
df_dev_sample.loc[:,"reply_CoT"] = replies

In [ ]:
#let's look at the result:
df_dev_sample["reply_CoT"].iloc[0]

In [ ]:
df_dev_sample.head()

We'll need a different is_similar function now, to get the correct string from the long answer.

In [ ]:
def is_similar_cot(gold_answer, gold_answer_idx, answer):
        if "correct answer is" not in answer:
              answer = answer.split(")")[-1]
        answer = answer.split("correct answer is")[-1]
        if len(answer) == 1:
            return 1 if gold_answer_idx  == answer else 0
        gold_answer = gold_answer.lower()
        answer = answer.lower()
        return 1 if gold_answer in  answer or answer in gold_answer else 0

In [ ]:
df_dev_sample["is correct reply CoT"] = df_dev_sample.apply(lambda row:is_similar_cot(row["answer"], row["answer_idx"], row["reply_CoT"]), axis=1)
df_dev_sample["is correct reply CoT"].value_counts(normalize=True)

It got worse :-(
Let's create an example! Note what dataset we're using...

In [ ]:
from tech_train_functions import create_example_CoT
question,answer,options = df_train.iloc[0]["question"],df_train.iloc[0]["answer"],df_train.iloc[0]["options"]
system_message_explainer = "Please explain step by step how to answer this question.\
Start with 'let's take it step by step' and end with 'the correct answer is:' and then the correct answer idx and the correct answer.\
First examine the case, and suggest treatments without considering the 4 options you got. Only then, return to the options and decide wich is best"

example = create_example_CoT(system_message_explainer,question,options,answer, client=client)
print(df_train.iloc[0]["question"])
print(example[1])


In [ ]:
print(df_train.iloc[0]["question"])
print(example)

Running again with CoT example:

In [ ]:
system_message_CoT = "You're a medical expert answering medical questions. Please answer step by step. Start with 'let's take it step by step'\
and end with 'the correct answer is:' and then the correct answer. Please allways end with this phrase"

sample_list = (df_dev_sample["question"]+" "+
                df_dev_sample["options"].astype(str)).tolist()
print("Input:{}".format(sample_list[0]))
print("Example:{}".format(example[1]))


replies = []
for q in tqdm(sample_list):
    ans = call_open_ai(system_message_CoT, q, examples=[example], client=client)
    replies.append(ans)
df_dev_sample.loc[:,"reply_CoT"] = replies

In [ ]:
df_dev_sample

Now let's check if we improved:

In [ ]:
df_dev_sample["is correct reply CoT"] = df_dev_sample.apply(lambda row:is_similar_cot(row["answer"], row["answer_idx"], row["reply_CoT"]), axis=1)
df_dev_sample["is correct reply CoT"].value_counts(normalize=True)

Better! See if you can improve Further by changing CoT prompt or adding additional examples. Don't forget to re-create the example accordingly. You can also try adding more examples and see if it improves the result.

In [ ]:
#For test time:

question = <insert question here>
options = <insert question here>
q = question+ " "+str(options)
call_open_ai(system_message_CoT, q, examples=[example], client=client)
